In [1]:
import pytraj as pt
import parmed as pmd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import networkx as nx
import scipy
import pickle as pkl
from tqdm.notebook import tqdm
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy import exp
from kneed import KneeLocator as KneedLoc
from sklearn.decomposition import PCA

In [2]:
plt.rcParams['font.family'] = 'Helvetica'

<h1> Graph Normalization and Thresholding </h1>

In [3]:
def compute_normalized_graph(matrix):
    row_sums = matrix.sum(axis=1)
    size = matrix.shape[0]
    new_matrix = np.zeros((size,size))
    for x,row in enumerate(matrix):
        #for every column's row
        for y, val in enumerate(row):
            #for every value in row
            current = val / row_sums[x] if row_sums[x] != 0 else 0
            new_matrix[x,y] = max([current,new_matrix[y,x]])
            new_matrix[y,x] = max([current,new_matrix[x,y]])

    return new_matrix

In [4]:
def threshold_normalized_graph(matrix, cutoff):
    size = matrix.shape[0]
    new_matrix = np.zeros((size,size))
    for x,row in enumerate(matrix):
        for y, val in enumerate(row):
            new_matrix[x,y] = 1 if matrix[x,y] > cutoff else 0

    return new_matrix

<h1> Pairwise Energy Tensor Analysis </h1>

In [5]:
def thresh_and_norm_matrix(tensor,channel,threshold):
    #same function as above, just quicker to access here outside of the tensor class
    '''
    channel (int): the energy channel to compute for
    threshold (float): the cutoff value

    returns : (numpy.ndarray) matrix of matrices summed over common edges, normalized,
    and thresholded
    '''
    energy_tensor = torch.Tensor(tensor[:,channel])
    sum_matrix = torch.sum(energy_tensor, dim=0).numpy()
    normalized = compute_normalized_graph(sum_matrix)
    #thresholded is binary
    thresholded = threshold_normalized_graph(normalized,threshold)
    weighted_threshold = thresholded * normalized
    return (thresholded,weighted_threshold)

<h1> Heat Kernel Analysis:</h1>

### Heat Kernel PCA (See: https://link.springer.com/chapter/10.1007/11815921_18)

In [6]:
def matrix_exponentiation(eigenvectors,eigenvalues,t):
    '''
    Raise e^matrix where matrix is a square
    '''
    shape = eigenvectors.shape[0]
    processed_eigenvalues = np.exp(-t*eigenvalues)
    e_diagonal = np.diag(processed_eigenvalues)
    return eigenvectors.dot(e_diagonal).dot(eigenvectors.transpose())

def get_heat_kernel(matrix,t):
    laplacian = scipy.sparse.csgraph.laplacian(matrix,normed=True)
    (eigenvalues,eigenvectors) = np.linalg.eigh(laplacian)
    x = matrix_exponentiation(eigenvectors,eigenvalues,t)
    x = np.round(x, decimals = 6)
    return x

def kernel_from_energy_tensor_thresh(tensor,threshold,time):
    kernels = np.zeros((tensor.shape[0], tensor.shape[2], tensor.shape[2]))
    for i, matrix in enumerate(tensor):
        normalized = compute_normalized_graph(matrix)
        thresholded = threshold_normalized_graph(normalized,threshold)
        final_mat = thresholded * normalized
        kernel = get_heat_kernel(final_mat, time)
        kernels[i] = kernel

    return kernels

In [7]:
def get_pca(kernel,dim):
    pca = PCA(n_components=dim)
    pca.fit(kernel)
    transformed = pca.transform(kernel)
    return (transformed, pca.explained_variance_)

In [8]:
def make_shared_pca_df(kernels, num_components, show_var=False):
    dfs = []
    eigens=[]
    for kernel in kernels:
        (pcs, var) = get_pca(kernel, num_components)
        df = pd.DataFrame(pcs)
        dfs.append(df)
        if show_var:
            eigens+=[var]

    return (eigens, pd.concat(dfs))

In [9]:
#assigns res number to each pcs embedding
def embedded_res_assign(frame_tot,res_tot,df):
    lst=np.zeros(frame_tot*res_tot)
    n=res_tot
    res_idx=[]
    i=1
    res=1
    while i<=len(lst):
        if res>res_tot:
            res=1
            res_idx.append(res)
        else:
            res_idx.append(res)
        res+=1
        i+=1
    df['res'] =res_idx
    return df

def embedded_frame_assign(frame_tot,res_tot,df):
    lst=np.zeros(frame_tot*res_tot)
    n=res_tot
    frame_idx=[]
    i=1
    frame=0
    while i<=len(lst):
        if (i)%n==0:
            frame_idx.append(frame)
            frame+=1
        else:
            frame_idx.append(frame)
        i+=1
    df['frame'] =frame_idx
    return df


def heat_map(kernel,pcs):
    diag_els=[]
    for matrix in kernel[:]:
        diag_element=np.diag(matrix)
        diag_els+=[diag_element]
    diag_flat=np.concatenate(diag_els)
    #set the column called 'heat' to the flattened diagonal matrix consisting of the diagonal elements of the kernel
    pcs['heat'] =diag_flat
    return pcs

#range_color1=[0.0112,0.0134]
range_color1=[]
xrng=[-0.013, 0.024]
yrng=[-0.013, 0.015]
def heat_map_plot(kernel,pcs,pc_b,title_name):
    heat_pcs=heat_map(kernel,pcs)
    heat_pcs_res0=embedded_res_assign(100,199,heat_pcs)
    heat_pcs_res=embedded_frame_assign(100,199,heat_pcs)
    fig=px.scatter(heat_pcs_res, x= 0, y= pc_b, color="heat",range_color=range_color1,hover_data=["heat","res","frame"], labels={ "0": "PC1","2": "PC3"},title=title_name)
    fig.update_layout(title_x=0.5)
    fig.update_layout(title_yanchor="top")
    fig.update_traces(marker=dict(size=10))
    fig.update_xaxes(range=xrng)
    fig.update_yaxes(range=yrng)
    fig.update_layout(font=dict(
        family="Arial",
        size=16))
    fig.show()

def three_dim_heat_map_plot(kernel,pcs):
    heat_pcs=heat_map(kernel,pcs)
    heat_pcs_res0=embedded_res_assign(100,199,heat_pcs)
    heat_pcs_res=embedded_frame_assign(100,199,heat_pcs)
    fig=px.scatter_3d(pcs, x=0, y=1, z=2,color="heat",hover_data=["heat","res","frame"])
    fig.update_traces(marker_size = 3)
    fig.show()

def heat_map_plot_resid(kernel,pcs,pc_b):
    heat_pcs=heat_map(kernel,pcs)
    heat_pcs_res=embedded_res_assign(100,199,heat_pcs)
    fig=px.scatter(heat_pcs_res, x= 0, y= pc_b, color="res",color_continuous_scale=px.colors.sequential.Turbo)
    fig.update_traces(marker=dict(size=10))

    fig.show()

def heat_res_frame_map(kernel,pcs,num_frame,num_res):
    heat_pcs=heat_map(kernel,pcs)
    heat_pcs_res=embedded_res_assign(num_frame,num_res,heat_pcs)
    heat_pcs_res_frame=embedded_frame_assign(num_frame,num_res,heat_pcs)
    return heat_pcs_res_frame



# Energy Tensors:

In [10]:
#etensor indexing: frame, row, column

In [11]:
def LoadData(file):
    toReturn = []
    with np.load(file) as data:
        lst = data.files
        for item in lst:
            toReturn.append(data[item])
    if len(toReturn) > 1: raise ValueError(f"file has length {len(toReturn)}")
    return toReturn[0]

In [12]:
# wt_etensor=np.load('/home/student5/Desktop/Energetics/elec_1.npz')
# for x in wt_etensor:
#     print(x)

path = "/home/student5/Desktop/Energetics/zfshomes/sstetson/End_Fall23-Pres/Analysis/DBD/ff14SB/Energetics/"
etensor = [
    LoadData(path + f"WT_PK11000/Rep1/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"WT_PK11000/Rep1/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"WT_PK11000/Rep1/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"WT_PK11000/Rep1/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"WT_PK11000/Rep1/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
] + [
    LoadData(path + f"WT_PK11000/Rep2/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"WT_PK11000/Rep2/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"WT_PK11000/Rep2/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"WT_PK11000/Rep2/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"WT_PK11000/Rep2/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
] + [
    LoadData(path + f"WT_PK11000/Rep3/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"WT_PK11000/Rep3/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"WT_PK11000/Rep3/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"WT_PK11000/Rep3/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"WT_PK11000/Rep3/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
] + [
    LoadData(path + f"WT_PK11000/Rep4/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"WT_PK11000/Rep4/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"WT_PK11000/Rep4/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"WT_PK11000/Rep4/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"WT_PK11000/Rep4/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
]

etensor = np.array(etensor)
etensor

array([[[0.        , 0.01904599, 0.01353679, ..., 0.00376342,
         0.0043243 , 0.0047106 ],
        [0.01904599, 0.        , 0.01645381, ..., 0.00343131,
         0.0039511 , 0.00424955],
        [0.01353679, 0.01645381, 0.        , ..., 0.00323744,
         0.00368101, 0.00392055],
        ...,
        [0.00376342, 0.00343131, 0.00323744, ..., 0.        ,
         0.02651854, 0.0210984 ],
        [0.0043243 , 0.0039511 , 0.00368101, ..., 0.02651854,
         0.        , 0.02971518],
        [0.0047106 , 0.00424955, 0.00392055, ..., 0.0210984 ,
         0.02971518, 0.        ]],

       [[0.        , 0.01958412, 0.01401863, ..., 0.00328086,
         0.0036986 , 0.00407919],
        [0.01958412, 0.        , 0.01554034, ..., 0.00322682,
         0.00364579, 0.00403168],
        [0.01401863, 0.01554034, 0.        , ..., 0.00305132,
         0.0034104 , 0.00374181],
        ...,
        [0.00328086, 0.00322682, 0.00305132, ..., 0.        ,
         0.0274453 , 0.02170999],
        [0.0

### Determining Time and Thresh Parameters

In [13]:
time = 5
thresh = 0.003
num_of_pcs = 5

kernel = kernel_from_energy_tensor_thresh(etensor,thresh,time)
eigens, pcs = make_shared_pca_df(kernel,num_of_pcs,show_var=True)

eigens_df = pd.DataFrame(eigens)
mean_n_eigens = eigens_df.mean(axis=0)

## PCA Embedding Heat Kernels:

In [14]:
#getting the principle components from the heatkernel
#changed the last argument to 2 and it changed stuff. Ask Dylan how many "components" I should be doing (2 or 3)? We want 3 typically
(_,pcs) = make_shared_pca_df(kernel,3)

## System Heat Kernel Shared PCA Plots

In [15]:
heat = heat_res_frame_map(kernel, pcs, 4000, 194)

In [16]:
heat.to_csv("/home/student5/Desktop/Energetics/Processed/DBD_WT_PK11000_Long_Elec_1-194.csv")

In [17]:
# wt_etensor=np.load('/home/student5/Desktop/Energetics/elec_1.npz')
# for x in wt_etensor:
#     print(x)

path = "/home/student5/Desktop/Energetics/zfshomes/sstetson/End_Fall23-Pres/Analysis/DBD/ff14SB/Energetics/"
etensor = [
    LoadData(path + f"Y220C/Rep1/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"Y220C/Rep1/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"Y220C/Rep1/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"Y220C/Rep1/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"Y220C/Rep1/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
] + [
    LoadData(path + f"Y220C/Rep2/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"Y220C/Rep2/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"Y220C/Rep2/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"Y220C/Rep2/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"Y220C/Rep2/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
] + [
    LoadData(path + f"Y220C/Rep3/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"Y220C/Rep3/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"Y220C/Rep3/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"Y220C/Rep3/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"Y220C/Rep3/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
] + [
    LoadData(path + f"Y220C/Rep4/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"Y220C/Rep4/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"Y220C/Rep4/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"Y220C/Rep4/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"Y220C/Rep4/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
]

etensor = np.array(etensor)
etensor

### Determining Time and Thresh Parameters

time = 5
thresh = 0.003
num_of_pcs = 5

kernel = kernel_from_energy_tensor_thresh(etensor,thresh,time)
eigens, pcs = make_shared_pca_df(kernel,num_of_pcs,show_var=True)

eigens_df = pd.DataFrame(eigens)
mean_n_eigens = eigens_df.mean(axis=0)

## PCA Embedding Heat Kernels:

#getting the principle components from the heatkernel
#changed the last argument to 2 and it changed stuff. Ask Dylan how many "components" I should be doing (2 or 3)? We want 3 typically
(_,pcs) = make_shared_pca_df(kernel,3)

## System Heat Kernel Shared PCA Plots

heat = heat_res_frame_map(kernel, pcs, 4000, 194)

heat.to_csv("/home/student5/Desktop/Energetics/Processed/DBD_Y220C_Long_Elec_1-194.csv")

In [18]:
# wt_etensor=np.load('/home/student5/Desktop/Energetics/elec_1.npz')
# for x in wt_etensor:
#     print(x)

path = "/home/student5/Desktop/Energetics/zfshomes/sstetson/End_Fall23-Pres/Analysis/DBD/ff14SB/Energetics/"
etensor = [
    LoadData(path + f"Y220C_PK11000/Rep1/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"Y220C_PK11000/Rep1/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"Y220C_PK11000/Rep1/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"Y220C_PK11000/Rep1/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"Y220C_PK11000/Rep1/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
] + [
    LoadData(path + f"Y220C_PK11000/Rep2/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"Y220C_PK11000/Rep2/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"Y220C_PK11000/Rep2/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"Y220C_PK11000/Rep2/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"Y220C_PK11000/Rep2/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
] + [
    LoadData(path + f"Y220C_PK11000/Rep3/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"Y220C_PK11000/Rep3/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"Y220C_PK11000/Rep3/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"Y220C_PK11000/Rep3/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"Y220C_PK11000/Rep3/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
] + [
    LoadData(path + f"Y220C_PK11000/Rep4/1-200/normal/elec_1-194/elec_{x}.npz") for x in range(1, 201)
] + [
    LoadData(path + f"Y220C_PK11000/Rep4/201-400/normal/elec_1-194/elec_{x}.npz") for x in range(201, 401)
] + [
    LoadData(path + f"Y220C_PK11000/Rep4/401-600/normal/elec_1-194/elec_{x}.npz") for x in range(401, 601)
] + [
    LoadData(path + f"Y220C_PK11000/Rep4/601-800/normal/elec_1-194/elec_{x}.npz") for x in range(601, 801)
] + [
    LoadData(path + f"Y220C_PK11000/Rep4/801-1000/normal/elec_1-194/elec_{x}.npz") for x in range(801, 1001)
]

etensor = np.array(etensor)
etensor

### Determining Time and Thresh Parameters

time = 5
thresh = 0.003
num_of_pcs = 5

kernel = kernel_from_energy_tensor_thresh(etensor,thresh,time)
eigens, pcs = make_shared_pca_df(kernel,num_of_pcs,show_var=True)

eigens_df = pd.DataFrame(eigens)
mean_n_eigens = eigens_df.mean(axis=0)

## PCA Embedding Heat Kernels:

#getting the principle components from the heatkernel
#changed the last argument to 2 and it changed stuff. Ask Dylan how many "components" I should be doing (2 or 3)? We want 3 typically
(_,pcs) = make_shared_pca_df(kernel,3)

## System Heat Kernel Shared PCA Plots

heat = heat_res_frame_map(kernel, pcs, 4000, 194)

heat.to_csv("/home/student5/Desktop/Energetics/Processed/DBD_Y220C_PK11000_Long_Elec_1-194.csv")